 # SQL Playground

 This notebook is used to:
 - populate the database with sample data
 - run basic SQLAlchemy queries
 - experiment with joins and aggregations

 Stored in Jupytext percent format.

 ## Imports

 Import SQLAlchemy helpers and local database / ORM setup.

In [1]:
from sqlalchemy import select, func

from db import SessionLocal
from models import Category, Product

 ## Open a database session

 A session represents a unit of work against the database.
 All reads and writes go through this session.

In [2]:
session = SessionLocal()

 ## Optional clean start

 Remove existing rows so the notebook can be executed repeatedly
 with predictable results.

In [3]:
session.query(Product).delete()
session.query(Category).delete()
session.commit()

2025-12-16 20:13:49,430 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-16 20:13:49,432 INFO sqlalchemy.engine.Engine DELETE FROM products
2025-12-16 20:13:49,433 INFO sqlalchemy.engine.Engine [generated in 0.00076s] ()
2025-12-16 20:13:49,434 INFO sqlalchemy.engine.Engine DELETE FROM categories
2025-12-16 20:13:49,435 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ()
2025-12-16 20:13:49,436 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
# session.close()

 ## Insert categories

 Categories must exist before products because products
 reference them via a foreign key.

In [5]:
electronics = Category(name="electronics")
books = Category(name="books")
food = Category(name="food")

session.add_all([electronics, books, food])
session.commit()

2025-12-16 20:13:49,456 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-16 20:13:49,458 INFO sqlalchemy.engine.Engine INSERT INTO categories (name, created_at) VALUES (?, ?) RETURNING id
2025-12-16 20:13:49,458 INFO sqlalchemy.engine.Engine [generated in 0.00010s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('electronics', '2025-12-16 19:13:49.458067')
2025-12-16 20:13:49,459 INFO sqlalchemy.engine.Engine INSERT INTO categories (name, created_at) VALUES (?, ?) RETURNING id
2025-12-16 20:13:49,460 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('books', '2025-12-16 19:13:49.458070')
2025-12-16 20:13:49,460 INFO sqlalchemy.engine.Engine INSERT INTO categories (name, created_at) VALUES (?, ?) RETURNING id
2025-12-16 20:13:49,461 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('food', '2025-12-16 19:13:49.458071')
2025-12-16 20:13:49,461 INFO sqlalchemy.engine.Engine COMMIT


 ## Insert products

 Each product references an existing category by its ID.

In [6]:
products = [
    Product(name="Laptop", price=1200.0, category_id=electronics.id),
    Product(name="Headphones", price=199.0, category_id=electronics.id),
    Product(name="Python Book", price=39.9, category_id=books.id),
    Product(name="Bread", price=3.5, category_id=food.id),
]

session.add_all(products)
session.commit()

2025-12-16 20:13:49,472 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-16 20:13:49,473 INFO sqlalchemy.engine.Engine SELECT categories.id AS categories_id, categories.name AS categories_name, categories.created_at AS categories_created_at 
FROM categories 
WHERE categories.id = ?
2025-12-16 20:13:49,474 INFO sqlalchemy.engine.Engine [generated in 0.00065s] (1,)
2025-12-16 20:13:49,476 INFO sqlalchemy.engine.Engine SELECT categories.id AS categories_id, categories.name AS categories_name, categories.created_at AS categories_created_at 
FROM categories 
WHERE categories.id = ?
2025-12-16 20:13:49,476 INFO sqlalchemy.engine.Engine [cached since 0.0028s ago] (2,)
2025-12-16 20:13:49,477 INFO sqlalchemy.engine.Engine SELECT categories.id AS categories_id, categories.name AS categories_name, categories.created_at AS categories_created_at 
FROM categories 
WHERE categories.id = ?
2025-12-16 20:13:49,477 INFO sqlalchemy.engine.Engine [cached since 0.004158s ago] (3,)
2025-12-16 20:13:4

 ## Select all products

 Simple SELECT to verify that data exists.

In [7]:
session.execute(
    select(Product)
).all()

2025-12-16 20:13:49,494 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-16 20:13:49,495 INFO sqlalchemy.engine.Engine SELECT products.id, products.name, products.price, products.in_stock, products.category_id, products.created_at 
FROM products
2025-12-16 20:13:49,495 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ()


[(<models.Product object at 0x000001D94B6B3A10>,),
 (<models.Product object at 0x000001D94B70A350>,),
 (<models.Product object at 0x000001D94B70A850>,),
 (<models.Product object at 0x000001D94B672650>,)]

 ## Filter products by price

 Return only products cheaper than 100.

In [8]:
session.execute(
    select(Product.name, Product.price)
    .where(Product.price < 100)
).all()

2025-12-16 20:13:49,506 INFO sqlalchemy.engine.Engine SELECT products.name, products.price 
FROM products 
WHERE products.price < ?
2025-12-16 20:13:49,507 INFO sqlalchemy.engine.Engine [generated in 0.00050s] (100,)


[('Python Book', 39.9), ('Bread', 3.5)]

 ## Join products with categories

 Combine data from both tables using an explicit JOIN.

In [9]:
session.execute(
    select(
        Product.name,
        Category.name.label("category"),
        Product.price,
    )
    .join(Category, Product.category_id == Category.id)
).all()

2025-12-16 20:13:49,517 INFO sqlalchemy.engine.Engine SELECT products.name, categories.name AS category, products.price 
FROM products JOIN categories ON products.category_id = categories.id
2025-12-16 20:13:49,517 INFO sqlalchemy.engine.Engine [generated in 0.00060s] ()


[('Laptop', 'electronics', 1200.0),
 ('Headphones', 'electronics', 199.0),
 ('Python Book', 'books', 39.9),
 ('Bread', 'food', 3.5)]

 ## Count products per category

 Aggregation using GROUP BY.

In [10]:
session.execute(
    select(
        Category.name,
        func.count(Product.id).label("product_count"),
    )
    .join(Product, Product.category_id == Category.id)
    .group_by(Category.name)
).all()

2025-12-16 20:13:49,527 INFO sqlalchemy.engine.Engine SELECT categories.name, count(products.id) AS product_count 
FROM categories JOIN products ON products.category_id = categories.id GROUP BY categories.name
2025-12-16 20:13:49,528 INFO sqlalchemy.engine.Engine [generated in 0.00070s] ()


[('books', 1), ('electronics', 2), ('food', 1)]

 ## Average price per category

 Aggregation example using AVG.

In [11]:
session.execute(
    select(
        Category.name,
        func.avg(Product.price).label("avg_price"),
    )
    .join(Product)
    .group_by(Category.name)
).all()

2025-12-16 20:13:49,537 INFO sqlalchemy.engine.Engine SELECT categories.name, avg(products.price) AS avg_price 
FROM categories JOIN products ON categories.id = products.category_id GROUP BY categories.name
2025-12-16 20:13:49,538 INFO sqlalchemy.engine.Engine [generated in 0.00053s] ()


[('books', 39.9), ('electronics', 699.5), ('food', 3.5)]

 ## Cleanup

 Close the session when finished working with the database.

In [12]:
session.close()

2025-12-16 20:13:49,547 INFO sqlalchemy.engine.Engine ROLLBACK
